XGBoost Regression Modeling

In [15]:
import pickle
import pathlib
import numpy as np
import pandas as pd
import xgboost as xgb

In [16]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)
clean_data_path = DATA_DIR / 'processed' / 'ames_clean_eng.pkl'
with open(clean_data_path, 'rb') as file:
    data = pickle.load(file)

/Users/femcdias/Documents/Insper/Semestre 4/Machine Learning/Projeto1/ames/data


In [17]:
y = data['SalePrice'].copy()
X = data.drop(columns=['SalePrice']).copy()

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
# XGBoost model setup
model = xgb.XGBRegressor(objective ='reg:squarederror')
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [21]:
ypred = model.predict(X_test)

In [22]:
from sklearn.metrics import mean_squared_error
RMSE = np.sqrt(mean_squared_error(y_test, ypred))
print(f'RMSE: {RMSE}')
error_percentage = 100 * (10**RMSE - 1)
print(f'Average error is {error_percentage:.2f}%')

RMSE: 0.05686298779156216
Average error is 13.99%


In [23]:
from sklearn.model_selection import GridSearchCV
model = xgb.XGBRegressor(objective ='reg:squarederror')

# Already tested, best parameters are {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 5, 'n_estimators': 150, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 0.7}
# parameters = {
#     'max_depth': [3, 6, 9],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'n_estimators': [50, 100, 150],
#     'gamma': [0, 0.2, 0.4],
#     'min_child_weight': [1, 3, 5],
#     'subsample': [0.7, 0.9],
#     'colsample_bytree': [0.7, 0.9],
#     'reg_alpha': [0, 0.1],
#     'reg_lambda': [1, 1.5]
# }

parameters = {'colsample_bytree': [0.7], 'gamma': [0], 'learning_rate': [0.05], 'max_depth': [6], 'min_child_weight': [5], 'n_estimators': [150], 'reg_alpha': [0], 'reg_lambda': [1], 'subsample': [0.7]}

grid_search = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

best_params_xgb = grid_search.best_params_
print(best_params_xgb)
best_model_xgb = grid_search.best_estimator_

/Users/femcdias/Documents/Insper/Semestre 4/Machine Learning/Projeto1/ames/env/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/femcdias/Documents/Insper/Semestre 4/Machine Learning/Projeto1/ames/env/lib/python3.11/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Users/femcdias/Documents/Insper/Semestre 4/Machine Learning/Projeto1/ames/env/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Users/femcdias/Documents/Insper/Semestre 4/Machine Learning/Projeto1/ames/env/

{'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 5, 'n_estimators': 150, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 0.7}


In [24]:
y_pred_xgb = best_model_xgb.predict(X_test)
RMSE_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print(f'RMSE: {RMSE_xgb}')
error_percentage_xgb = 100 * (10**RMSE_xgb - 1)
print(f'Average error is {error_percentage_xgb:.2f}%')

RMSE: 0.049675380054110256
Average error is 12.12%
